# Feature Selection with the D-Wave System
Placeholder sentence: In the [feature selection demo](<<< leap_url >>>/demos/socialnetwork/) you saw ...   

This notebook examines how you can solve optimization problems on a  D-Wave quantum processing unit (QPU) with the example of a feature-selection problem.
    
1. [What is Feature Selection?](#What-is-Feature-Selection?) defines and explains the feature-selection problem.
2. [Solving Feature Selection on a Quantum Computer](#Solving-Feature-Selection-on-a-Quantum-Computer) shows how such optimization problems can be formulated for solution on a quantum computer. 
3. [Example Application: Predicting Survival of Titanic Passangers](#Example-Application:-Predicting-Survival-of-Titanic-Passangers) uses feature selection on a known problem. 
4. [Feature Selection and Mutual Information](#Feature-Selection-and-Mutual-Information) provides more details on the mathematics used by this example.

This notebook should help you understand both the techniques and [Ocean software](https://github.com/dwavesystems) tools used for solving optimization problems on D-Wave quantum computers.

**New to Jupyter Notebooks?** JNs are divided into text or code cells. Pressing the **Run** button in the menu bar moves to the next cell. Code cells are marked by an "In: \[\]" to the left; when run, an asterisk displays until code completion: "In: \[\*\]".

# What is Feature Selection?
Statistical and machine-learning models use a set of input variables (features)
to predict output variables of interest. Feature selection, which can be
part of the model design process, simplifies the model and reduces dimensionality by selecting,
from a given set of potential features, a subset of highly informative ones. 

For example, if Farmer Jones were creating a model for predicting the ripening of her hothouse tomatoes, she might start recording daily the following list of potential features: date, air temperature, degree of cloudiness, hours of daylight, daily water, fertilizer, air humidity, hours of electric light, ambient music style. After a growth season or two, she analyzes correlations between these features and her tomato crops. Her analysis reveals:

* date, cloudiness, daylight have little predictive power
* water and humidity are highly predictive of crop rot; they are also highly correlated (the hothouse has a roof sprinkler) 
* fertilizer is highly predictive of fruit size

Farmer Jones understands that her hothouse's electric light makes her crop less dependant on seasons (date) and sunshine (cloudiness). She can simplify her model by disregarding those features. She can also reduce the number of inputs by recording either her water or humidity measurement but not both.

For systems with large amounts of potential input information, such as weather forecasting or facial recognition, the model complexity and required compute resources can be daunting. Feature selection can help make such models tractable. 

However, optimal feature selection itself can be a hard problem. This example introduces a powerful method of optimizing feature selection based on a hard probability calculation. To overcome the difficulties of this calculation, it formulates a solution by quantum computer.  

## Illustrative Toy Problem
This section illustrates the use of feature selection with a very simple example.

An example system has a single output that is generated by three inputs. The model then used to predict the output is even simpler than the system, with just two features. If all three inputs contribut somewhat commensurately* to the output, the model should perform better if the selected features are more independant. If this was a model of Farmer Jones's tomatoes, selecting water and fertilizer should be better than water and humidity.

*: If an independant input contributes less than the difference between two correlated ones, this might not be true. 

The next cell creates three inputs, two of which are very similar---a sine and a noisy sine---and a random one.  For simplicity, the output is a linear combination of the three inputs. 

In [ ]:
import numpy as np
import pandas as pd

# Three features: in1 & in2 are similar; in3 is different 
in1 = np.sin(np.linspace(-np.pi, np.pi, 100)).reshape(100,1)
in2 = np.sin(np.linspace(-np.pi+0.1, np.pi+0.2, 100)).reshape(100,1) + 0.3*np.random.rand(100, 1)
in3 = np.linspace(-1, 1, 100).reshape(100,1).reshape(100,1) + 2*np.random.rand(100, 1)

# Variable of interest's relationship to the features
out = 2*in1 + 3*in2 + 6*in3

# Store all in a pandas DataFrame for later use
toy = pd.DataFrame(np.hstack((in1, in2, in3, out)), columns=["in1", "in2", "in3", "out"])

The next cell plots the features (three inputs) and the variable of interest (the output). You can see that inputs 1 and 2 are much more correlated to each other than either of those two are correlated to input 3.

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
from helpers.plots import subplot_xx_y
subplot_xx_y(toy)

The next cell defines a simple linear model of only two variables. It uses SciPy's `curve_fit` function to try to reproduce the variable of interest using first the two correlated features (inputs 1 and 2) and then two disimilar features (inputs 1 and 3). It prints for both models a measure of the statistical error in the fitting for comparison.

In [ ]:
from scipy.optimize import curve_fit

# Define a simple two-variable model
def two_var_model(IN, a, b):
    ina, inb = IN
    return a*ina + b*inb

popt_12, pcov_12 = curve_fit(two_var_model, (toy['in1'].values, toy['in2'].values), toy['out'].values) 
popt_13, pcov_13 = curve_fit(two_var_model, (toy['in1'].values, toy['in3'].values), toy['out'].values) 
 
perr_12 = max(np.sqrt(np.diag(pcov_12))) 
perr_13 = max(np.sqrt(np.diag(pcov_13))) 

print("Standard deviation is under {:.2f} for selection of features 1 & 2 and {:.2f} for features 1 & 3.".format(perr_12+0.01, perr_13+0.01))

The next cell plots comparison graphs between the outputs generated by both two-variable models and the output of the system. You can see that the second model, which uses two less correlated inputs, performs better.

In [ ]:
from helpers.plots import plot_vars_out
plot_vars_out(toy, ['in1', 'in2', popt_12], ['in1', 'in3', popt_13], out, two_var_model) 

The following sections can help you benefit from the advantages of optimal feature selection in modeling more complex systems.

## Feature Selection by Mutual Information
There are many methods to do feature selection. For example, if you are building a deep learning network and have six potential features, you might try training first on each of the features by itself, then on all 15 combinations of subsets of two features, then 20 combinations of subsets of three features, and so on. This naive method rarely works for real-world cases. For those, statistical methods are much more efficient.  

One statistical criterion that can guide this selection is mutual information (MI). Section [Feature Selection and Mutual Information](#Feature-Selection-and-Mutual-Information) describes the concepts and mathematics of this statistical tool in more detail. At a high level, MI quantifies how much one knows about one random variable from observations of another variable. $I(X;Y)$, the MI between variables $X$ and $Y$, is defined as 

$I(X;Y) = \sum_{y \in Y} \sum_{x \in X} p(x, y) \log \frac{p(x,y)}{p(x)p(y)}$,

where $p(x)$ and $p(y)$ are marginal probabilities of $X$ and $Y$, and $p(x,y)$ the joint probability. 

Intuitively, if two features have high MI, a model based on just one of the features should be closer to a model based on both compared to features with low MI.   
Ideally, to select the $k$ most relevant features of $n$ for a model, you could maximize $I({X_s}; Y)$, the MI between a set of $k$ features, $X_s$, and the output variable of interest, $Y$. This is a hard calculation because $n \choose k$ grows rapidly in real-world problems.


## MI Calculation for the Toy Problem


Section [Illustrative Toy Problem](#Illustrative-Toy-Problem) showed features you could see are correlated from looking at their graphs. This section quantifies that using MI.  

For reference, the first cell calculates the MI between a feature ('in1') and itself. The analytical value is given as follows:
$\begin{aligned} I(X;Y=X) 
& = \sum_{y \in Y} \sum_{x \in X} p(x, y) \log \frac{p(x,y)}{p(x)p(y)}|_{Y=X} \\
& = \sum_{x \in X} p(x) \log \frac{p(x)}{p(x)p(x)} \\
& = \sum_{x \in X} p(x) \log \frac{1}{p(x)} \\
& = - \sum_{x \in X} p(x) \log p(x) \end{aligned}$

The final formula might look familiar; it is Shannon Entropy, $H(X)$, a measure of information in a signal. 

The following cells calculate MI for the toy problem to (a) give a sense of how MI quantifies the intuitive understand from looking at the graphs and (2) develop the functions used in the following sections.

To calculate MI, this section defines a probablity function that takes the data of some variables (this would be training data in a machine learning context) and divides it into bins to map the probability of any data point (as a statistical event).

In [ ]:
def prob(dataset, max_bins=10):
    """Joint probability distribution P(X) for the given data."""

    # bin by the number of different values per feature
    num_rows, num_columns = dataset.shape
    bins = [min(len(np.unique(dataset[:, ci])), max_bins) for ci in range(num_columns)]

    prob, _ = np.histogramdd(dataset, bins)
    return prob / np.sum(prob)

In [ ]:
def shannon_entropy(p):
    """Shannon entropy H(X) is the sum of P(X)log(P(X)) for probabilty distribution P(X)."""
    p = p.flatten()
    return -sum(pi*np.log2(pi) for pi in p if pi)

In [ ]:
np.log2(5)
np.sum(np.diag(prob(np.hstack((in1.reshape(100,1), in1.reshape(100,1))), max_bins=5)*np.log2(prob(np.hstack((in1.reshape(100,1), in1.reshape(100,1))), max_bins=5))))

In [ ]:
p_11 = prob(np.hstack((in1.reshape(100,1), in1.reshape(100,1))), max_bins=5)
print(mutual_information(p_11, 0))

In [ ]:
dataset = np.hstack((in1.reshape(100,1), in1.reshape(100,1)))
num_rows, num_columns = dataset.shape
bins = [min(len(np.unique(dataset[:, ci])), 5) for ci in range(num_columns)]
p, edges = np.histogramdd(dataset, bins)

In [ ]:
np.histogramdd(in1.reshape(100,1), 5)
prob(in1.reshape(100,1), 5)

In [ ]:
print(shannon_entropy(prob(in1.reshape(100,1), 5)))

In [ ]:
dataset = np.hstack((in1.reshape(100,1), in1.reshape(100,1)))
mi_11 = mutual_information(prob(dataset, max_bins=5), 0)
print(mi_11)

In [ ]:
dataset = np.hstack((in1.reshape(100,1), in2.reshape(100,1)))
mi_12 = mutual_information(prob(dataset, max_bins=5), 0)
print(mi_12)

In [ ]:
dataset = np.hstack((in1.reshape(100,1), in3.reshape(100,1)))
mi_13 = mutual_information(prob(dataset, max_bins=5), 0)
print(mi_13)

# Solving Feature Selection on a Quantum Computer
There are different methods of approximating the hard calculation of $n \choose k$ features based on maximizing MI. One approach assumes conditional independence of the features and limits MI calculations to sets of three features. As shown in Section [Feature Selection and Mutual Information](#Feature-Selection-and-Mutual-Information), the optimal set of features is then given by:

$\frac{1}{k} \arg \max_s \sum_{i=1}^n \left \{ I(X_i;Y) + \sum_{j \in s|_i} I(X_j;Y |X_i) \right \}$


The lefthand component, $I(X_i;Y)$, represents MI between the output variable and a particular feature; maximizing means selecting features that are highly predictive of the output. The righthand component, $I(X_j;Y |X_i)$, represents conditional MI between the output variable and a feature given the selection of another feature; maximizing means selecting features that complement information about the output variable rather than provide redundant information.

This approximation is still a hard calculation; this section demonstrates a method for formulating it for solution on the D-Wave quantum computer. 

## MIQUBO
D-Wave systems solve binary quadratic models (BQM), the Ising model traditionally used in statistical mechanics and its computer-science equivalent, the quadratic unconstrained binary optimization (QUBO) problem. Given $N$ variables $x_1,...,x_N$, where each variable $x_i$ can have binary values $0$ or $1$, the system finds assignments of values that minimize,
    
$\sum_i^N q_ix_i + \sum_{i<j}^N q_{i,j}x_i  x_j$,
    
where $q_i$ and $q_{i,j}$ are configurable (linear and quadratic) coefficients. To formulate a problem for the D-Wave system is to program $q_i$ and $q_{i,j}$ so that assignments of $x_1,...,x_N$ also represent solutions to the problem.

For feature selection, the Mutual Information QUBO (MIQUBO) method formulates a QUBO based on the approximation above for $I({X_s}; Y)$, which is submitted to the D-Wave quantum computer for solution.



## Formulating a QUBO
The approximation above for optimal feature selection, with its limitation on calculating MI to sets of three variables, is a natural fit for QUBO formulation. 

<table style="width:70%">
  <tr>
    <th width="50%">Formula</th>
    <th width="10%">Optimization</th> 
    <th width="15%">Linear terms</th>
    <th width="15%">Quadratic terms</th>
  </tr>
  <tr>
      <td><b>Feature Selection</b>: $\sum_{i=1}^n \left \{ I(X_i;Y) + \sum_{j \in s|_i} I(X_j;Y |X_i) \right \}$</td>
    <td>Maximize</td> 
    <td>$I(X_i;Y)$</td>
    <td>$I(X_j;Y |X_i)$</td>
  </tr>
  <tr>
    <td><b>QUBO</b>: $\sum_i^N q_ix_i + \sum_{i<j}^N q_{i,j}x_i  x_j$</td>
    <td>Minimize</td> 
    <td>$q_ix_i$</td>
    <td>$q_{i,j}x_ix_j$</td>
  </tr>
</table>

Because the quantum computer seeks to minimize the programmed problem, these terms of the feature-selection formula should be set negative in the QUBO.   



To represent a choice of $k$ features out of a set of $n$, it makes sense that the assignments of $x_1,...,x_N$, which represent solutions to the problem, set $x_i$ to $1$ if feature $X_i$ should be selected and to $0$ if not. 

With solutions encoded this way, you can represent the QUBO in matrix format, 

$\mathbf{x}^T \mathbf{Q x}$

where $\mathbf Q$ is an $n$ X $n$ matrix and $\mathbf{x}$ is an $n$ x $1$ matrix (a vector) that should have $k$ ones representing the best features. To map the approximation above to a QUBO, the diagonal elements of $\mathbf Q$, representing linear coefficients, should minimize MI between the output and each feature, $Q_{ii} \leftarrow -I(X_i;Y)$, and non-diagonal elements, representing quadratic elements of $\mathbf Q$, should minimize for pairs of features that complement information about the output variable rather than provide redundant information, $Q_{ij} \leftarrow -I(X_j;Y |X_i)$.

At this point, the mapping does not include any requirement on the number of selected features. 

### Penalizing $len(s) != k$
How do you represent the requirement that only $k$ features be selected? By penalizing solutions with more or less than $k$ ones:

$P = \alpha \sum_{i=1}^n ( x_i - k)^2$ 

where $P$ is the penalty. 

# Example Application: Predicting Survival of Titanic Passangers
The demo illustrates the MIQUBO method by finding an optimal feature set for predicting
survival of Titanic passengers. It uses records provided in file
formatted_titanic.csv, which is a feature-engineered version of a public database of
passenger information recorded by the ship's crew (in addition to a column showing
survival for each passenger, it contains information on gender, title, class, port
of embarkation, etc). Its output is a ranking of subsets of features that have
high MI with the variable of interest (survival) and low redundancy.

# Summary

# Feature Selection and Mutual Information
This section explains the math of mutual information.

As described above, to select the :math:`k` most relevant features, you might maximize
:math:`I({X_s}; Y)`, the MI between a set of :math:`k` features, :math:`X_s`, and the
variable of interest, :math:`Y`. Given :math:`N` features out of which you select
:math:`k`, maximize mutual information, I, as

.. math::
    {X_1, X_2, ...X_k} = \arg \max I(X_k; Y)
by expanding,

.. math::
    I(X_k;Y) = N^{-1} \sum_i \left\{ I(X_i;Y) + I(X_{k(i)};Y|X_i) \right\}
Approximate the second term by assuming conditional independence:

.. math::
    I(X_k;Y|X_i) \approx \sum_{j \in X_k(i)} I(X_j;Y|X_i)
Using the following equations for Shannon entropy,

.. math::
    H(X) = -\sum_x P(x)\mathrm{log}P(x)
    H(X|Y) = H(X,Y)-H(Y)
You can then calculate all these terms as follows:

.. math::
     I(X;Y) = H(X)-H(X|Y)
     I(X;Y|Z) = H(X|Z)-H(X|Y,Z)
The approximated equation for MI can now be formed as a QUBO:

.. math:
    {X_1, X_2, ...X_k} = \arg \max \left\{MI - Penalty}
where the penalty is some multiple of :math:`\sum_{i} (x_i - k)^2` that enforces
the constraint of :math:`k` features.

In [ ]:
import itertools
import numpy as np
import pandas as pd

def prob(dataset, max_bins=10):
    """Joint probability distribution P(X) for the given data."""

    # bin by the number of different values per feature
    num_rows, num_columns = dataset.shape
    bins = [min(len(np.unique(dataset[:, ci])), max_bins) for ci in range(num_columns)]

    prob, _ = np.histogramdd(dataset, bins)
    return prob / np.sum(prob)

def shannon_entropy(p):
    """Shannon entropy H(X) is the sum of P(X)log(P(X)) for probabilty distribution P(X)."""
    p = p.flatten()
    return -sum(pi*np.log2(pi) for pi in p if pi)

def conditional_shannon_entropy(p, *conditional_indices):
    """Conditional Shannon entropy H(X;Y) = H(X,Y) - H(Y)."""

    axis = tuple(i for i in np.arange(len(p.shape)) if i not in conditional_indices)

    return shannon_entropy(p) - shannon_entropy(np.sum(p, axis=axis))

def mutual_information(prob, j):
    """Mutual information between variables X and variable Y.
    Calculated as I(X; Y) = H(X) - H(X|Y)."""

    return shannon_entropy(np.sum(prob, axis=j)) - conditional_shannon_entropy(prob, j)

def conditional_mutual_information(p, j, *conditional_indices):
    """Mutual information between variables X and variable Y conditional on variable Z.
    Calculated as I(X;Y|Z) = H(X|Z) - H(X|Y,Z)"""

    return conditional_shannon_entropy(np.sum(p, axis=j), *conditional_indices) - conditional_shannon_entropy(p, j, *conditional_indices)
